In [307]:
import tdgl
import light_sources.laguerre_gaussian_beam as lg
import inspect
import itertools
import logging
import math
import numbers
import os
from datetime import datetime
#from typing import Callable, Dict, List, NamedTuple, Optional, Sequence, Tuple, Union
import typing

import numpy as np
import scipy.sparse as sp

In [387]:
# Make a grid on the square film
a=2;b=0.1;xl=-a;xr=a;yb=-b;yt=b;
Nx=4;Ny=4;delx=(xr-xl)/(Nx-1);dely=(yt-yb)/(Ny-1);
xgrid=np.linspace(xl,xr,Nx)
ygrid=np.linspace(yb,yt,Ny)
#parameters of the SC
Gs=1;rs=-1;bs=1;as1=0.5;
Gp=1;rp=1.5;bp1=1;bp2=1;ap1=1;
asp1=asp2=0.25;bsp1=0.5;
# Parameters of the time-grid
Nt=1000;tau=1/Gs;dt=2*tau/Nt;

In [ ]:
# Boundary conditions: 

In [372]:
# Initial conditions:
psi=np.sqrt(-rs/bs)*np.ones([Nx, Ny])
Pxx=np.zeros([Nx, Ny])
mu=np.zeros([Nx, Ny])

In [337]:
# Vector potential:
def applied_vector_potential_x_fun(x,y,t):
    return Ax*np.cos(w*t)+0*np.cos(x)+0*np.sin(x)+1*np.sin(y)
t=0;w=1;Ax=0.5;
xg, yg = np.meshgrid(xgrid,ygrid)
Ax=np.asarray(applied_vector_potential_x_fun(xg,yg,t))
Ay=np.zeros((Nx,Ny));

In [338]:
lbx=np.zeros(Ny,dtype=np.int32);lby=np.arange(Ny,dtype=np.int32);
lboundary_indices=np.stack((lbx,lby),axis=1)
print(lboundary_indices)

[[0 0]
 [0 1]
 [0 2]
 [0 3]]


In [366]:
def dx(f: np.ndarray):
    fplusx=np.roll(f,-1,axis=1)
    return (fplusx-f)/delx

def dy(f: np.ndarray):
    fplusy=np.roll(f,-1,axis=0)
    return (fplusy-f)/dely

def dxx(f: np.ndarray):
    fplusx=np.roll(f,-1,axis=1)
    fminusx=np.roll(f,1,axis=1)
    return (fplusx-2*f+fminusx)/delx**2

def dyy(f: np.ndarray):
    fplusy=np.roll(f,-1,axis=0)
    fminusy=np.roll(f,1,axis=0)
    return (fplusy-2*f+fminusy)/dely**2

def Dxx(f: np.ndarray):
    return dxx(f)-1j*dx(Ax*f)-1j*Ax*dx(f)-(Ax**2)*f

def Dyy(f: np.ndarray):
    return dyy(f)-1j*dy(Ay*f)-1j*Ay*dy(f)-(Ay**2)*f

def Laplacian(f: np.ndarray):
    return Dxx(f)+Dyy(f)

In [390]:
def update(psi,Pxx):
    print(psi)
    #psiL=psi[0,:];psiR=psi[Nx-1,:];psiB=psi[:,0];psiT=psi[:,Ny-1];
    t=i*dt
    Ax=np.asarray(applied_vector_potential_x_fun(xg,yg,t))
    psi_sq=psi*np.conjugate(psi)
    psi=( (-rs-bs*psi_sq)*psi - as1*Laplacian(psi)-bsp1*Pxx*np.conjugate(Pxx)*psi-(asp1+asp2)*Dxx(Pxx) )*dt/Gs+psi
    #Pxx=( (-rp-(bp1+bp2)*Pxx*np.conjugate(Pxx))*Pxx - ap1*Laplacian(Pxx) - bsp1*psi_sq*Pxx-(asp1+asp2)*Dxx(psi) )*dt/Gp + Pxx
    psi[0,:]=psi[1,:]/(1+1j*Ax[0,:]*delx)
    psi[Nx-1,:]=psi[Nx-2,:]*(1+1j*Ax[Nx-2,:]*delx)
    psi[:,0]=psi[:,1]/(1+1j*Ay[:,0]*dely)
    psi[:,Ny-1]=psi[:,Ny-2]*(1+1j*Ay[:,Ny-2]*dely)
    #print(Pxx)
    return (psi,Pxx)

In [341]:
print(Ax)
print()
print(dyy(Ax)-1j*dy(Ay))
Dyy(Ax)

[[0.40016658 0.40016658 0.40016658 0.40016658]
 [0.46667284 0.46667284 0.46667284 0.46667284]
 [0.53332716 0.53332716 0.53332716 0.53332716]
 [0.59983342 0.59983342 0.59983342 0.59983342]]

[[ 5.98889450e+01+0.j  5.98889450e+01+0.j  5.98889450e+01+0.j
   5.98889450e+01+0.j]
 [ 3.33148193e-02+0.j  3.33148193e-02+0.j  3.33148193e-02+0.j
   3.33148193e-02+0.j]
 [-3.33148193e-02+0.j -3.33148193e-02+0.j -3.33148193e-02+0.j
  -3.33148193e-02+0.j]
 [-5.98889450e+01+0.j -5.98889450e+01+0.j -5.98889450e+01+0.j
  -5.98889450e+01+0.j]]


array([[ 5.98889450e+01+0.j,  5.98889450e+01+0.j,  5.98889450e+01+0.j,
         5.98889450e+01+0.j],
       [ 3.33148193e-02+0.j,  3.33148193e-02+0.j,  3.33148193e-02+0.j,
         3.33148193e-02+0.j],
       [-3.33148193e-02+0.j, -3.33148193e-02+0.j, -3.33148193e-02+0.j,
        -3.33148193e-02+0.j],
       [-5.98889450e+01+0.j, -5.98889450e+01+0.j, -5.98889450e+01+0.j,
        -5.98889450e+01+0.j]])

In [391]:
# Initial conditions:
psi=np.sqrt(-rs/bs)*np.ones([Nx, Ny])
Pxx=np.zeros([Nx, Ny])
mu=np.zeros([Nx, Ny])

In [392]:
for i in range(Nt):
    (psi,Pxx)=update(psi,Pxx)

[[1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]
 [1. 1. 1. 1.]]
[[0.86199248-0.34517991j 0.86199248-0.34517991j 0.86199248-0.34517991j
  0.86199248-0.34517991j]
 [1.00021778+0.j         1.00021778+0.j         1.00021778+0.j
  1.00021778+0.j        ]
 [1.00028444+0.j         1.00028444+0.j         1.00028444+0.j
  1.00028444+0.j        ]
 [1.00028444+0.755754j   1.00028444+0.755754j   1.00028444+0.755754j
  1.00028444+0.755754j  ]]
[[0.91577259-0.28904939j 0.91577259-0.28904939j 0.91577259-0.28904939j
  0.91577259-0.28904939j]
 [1.03152044+0.07766548j 1.03152044+0.07766548j 1.03152044+0.07766548j
  1.03152044+0.07766548j]
 [1.00058282-0.17004465j 1.00058282-0.17004465j 1.00058282-0.17004465j
  1.00058282-0.17004465j]
 [1.12905795+0.58593336j 1.12905795+0.58593336j 1.12905795+0.58593336j
  1.12905795+0.58593336j]]
[[0.99199765-0.18131836j 0.99199765-0.18131836j 0.99199765-0.18131836j
  0.99199765-0.18131836j]
 [1.06460477+0.21591713j 1.06460477+0.21591713j 1.06460477+0.21591713j
  1.0646047

In [383]:
print(list(range(Nt)))

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]


In [376]:
print(Pxx)

[[nan+nanj nan+nanj nan+nanj nan+nanj]
 [nan+nanj nan+nanj nan+nanj nan+nanj]
 [nan+nanj nan+nanj nan+nanj nan+nanj]
 [nan+nanj nan+nanj nan+nanj nan+nanj]]
